### 讀取大盤資料

In [ ]:
from finlab.data import Data

data = Data()

twii = data.get("發行量加權股價指數")

twii = twii['台股指數']

twii.head()

### 整理加權指數 1分k

In [ ]:

twii = twii[twii.index.second == 0]

### 畫的更美一點

In [ ]:
import matplotlib.pyplot as plt
plt.style.use
%matplotlib inline
twii.hist()

### DataFrame 小教室：groupby

In [ ]:
import pandas as pd

s = pd.Series([1,2,3,4,5,6])
s.groupby([0,1,0,1,0,1]).prod()

In [ ]:
df = pd.DataFrame({'a':s, 'b':s})
df
df.groupby([1,2,1,2,1,2]).prod()

In [ ]:
twii.head()

### 每月漲跌

In [ ]:
start_price = twii.groupby([twii.index.year, twii.index.month]).first()
last_price = twii.groupby([twii.index.year, twii.index.month]).last()
profit = (last_price/start_price)

# pandas <1.0.3
# profit.index.levels[0].name = 'year'
# profit.index.levels[1].name = 'month'

# pandas 1.0.3
profit.index=profit.index.set_names(['year','month'], level=[0,1])
profit = profit.reset_index()
profit = profit.pivot("year","month")#['台股指數']
profit
#profit

### 每月漲跌色溫圖

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (10, 10)

sns.heatmap(profit)

### 均線分析

In [ ]:
# 每日開盤價
twii_daily = twii.groupby(twii.index.date).last()
twii_daily.index = pd.to_datetime(twii_daily.index)

# 60日平均
twii_average = twii_daily.rolling(60).mean()

# 乖離率
twii_bias = twii_daily / twii_average

# 後60日漲跌
twii_profit = twii_daily.shift(-60) / twii_daily

# 將資料合併（方便之後做圖）
df = pd.DataFrame({
    'price': twii_daily,
    'sma': twii_average,
    'bias': twii_bias,
    'profit': twii_profit,
    'year': twii_daily.index.year,
})

# 刪除NaN
df = df.dropna(how='any')
df.head()

### 乖離率做圖

In [ ]:
sns.scatterplot(x="bias", y="profit", data=df.tail(200), hue="year", palette="Accent", legend="full")

### 查看當前的乖離率

In [ ]:
twii_bias.tail()